In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

# llm
from langchain import PromptTemplate, LLMChain
from langchain.llms import OpenAI

import os

from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA, ConversationalRetrievalChain

chat_template = """
나는 사용자의 직업을 추천해주는 챗봇이다. 

1. 이력서를 기반으로 저장된 이력서 파일과 유사도를 파악한다. 
2. 파악한 유사도를 기반으로 직업을 3가지 추천해준다. 
3. 이때 추천 직업별로 유사도 퍼센트를 포함하여 제공한다.
4. 직업추천 순서는 유사도가 높은 순서대로 제공한다. 
"""

chat_ai = ChatOpenAI(temperature=0.5, model="gpt-3.5-turbo")

c:\Users\hyeon\isic\2024-1-DSCD-ISIC-1\venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [3]:
# Retrieval QA Chain 생성
loader = CSVLoader("../../data/rallit_text.csv", encoding='utf8')
data = loader.load()

# Split the text in chunks, using LangChain Recursive Character Text Splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
    )

pages = loader.load_and_split(text_splitter)

# Create a persistent, file-based vector store, using Chroma vector store.
directory = 'data'
vector_index = Chroma.from_documents(
    pages, # Documents
    OpenAIEmbeddings(), # Text embedding model
    persist_directory=directory # persists the vectors to the file system
    )

vector_index.persist()

# Create the retriever and the query-interface.
retriever = vector_index.as_retriever(
    search_type="similarity", # Cosine Similarity
    search_kwargs={
        "k": 3, # Select top k search results
    }
)

qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature=0, model="gpt-4"),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True # source document which were used as source files
)


c:\Users\hyeon\isic\2024-1-DSCD-ISIC-1\venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
c:\Users\hyeon\isic\2024-1-DSCD-ISIC-1\venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [4]:
qa_chain.invoke("skill: python, java, kubernates, 프로젝트: 학교 웹사이트 개발에서 백엔드 담당, 데이터베이스 구축, 교육: 컴퓨터공학과.")

{'query': 'skill: python, java, kubernates, 프로젝트: 학교 웹사이트 개발에서 백엔드 담당, 데이터베이스 구축, 교육: 컴퓨터공학과.',
 'result': 'The user seems to be providing information rather than asking a question. However, if the user is asking if these skills and experiences are suitable for the job of a backend developer, then yes, the skills of Python, Java, and Kubernetes are relevant. Experience in project work such as backend responsibility in school website development and database construction is also valuable. Having an education in computer science is also beneficial for a backend developer role.',
 'source_documents': [Document(page_content='job: 백엔드 개발자\ntext: 기술 스택: Java, Spring, Spring Boot, MySQL, Python, JPA', metadata={'row': 494, 'source': '../../data/rallit_text.csv'}),
  Document(page_content='job: 백엔드 개발자\ntext: 기술 스택: Java, Spring, Spring Boot, MySQL, Python, JPA', metadata={'row': 494, 'source': '../../data/rallit_text.csv'}),
  Document(page_content='job: 백엔드 개발자\ntext: 기술 스택: Python, Django, 

In [5]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [8]:
# Retrieval QA Chain 생성
loader = CSVLoader("../../data/rallit_text.csv", encoding='utf8')
docs = loader.load()

# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

splits = text_splitter.split_documents(docs)

# 단계 3: 임베딩 & 벡터스토어 생성(Create Vectorstore)
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# 단계 4: 검색(Search)
# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

# 단계 5: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = hub.pull("rlm/rag-prompt")
'''
template = """
Answer the question based on the context below. You recommend a user's job.

1. 이력서를 기반으로 저장된 이력서 파일과 유사도를 파악한다. 
2. 파악한 유사도를 기반으로 직업을 3가지 추천해준다. 
3. 이때 추천 직업별로 유사도 퍼센트를 포함하여 제공한다.
4. 직업추천 순서는 유사도가 높은 순서대로 제공한다. 

Context: {context}
Question: {question} 

Answer: """


prompt_template = PromptTemplate(
    input_variables=["question"],
    template=template
)
'''



# 단계 6: 언어모델 생성(Create LLM)
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n".join(doc.page_content for doc in docs)


# 단계 7: 체인 생성(Create Chain)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 단계 8: 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "기술스택은 python, sql, java, kubernates이고, 프로젝트는 학교웹사이트를 개발한 적이 있습니다. 데이터베이스 구축과 백엔드 개발을 담당했습니다. 추천 직업을 알려주세요."
#question = "기술스택: python, sql, matplotlib, seaborn, selenium ,프로젝트: 카드사 고객 이탈 데이터 분석을 통한 아이디어 발굴 프로젝트. 크롤링을 통한 데이터 수집과 데이터 시각화, 예측 모델 설계를 담당했습니다. 추천 직업을 알려주세요"
response = rag_chain.invoke(question)

# 결과 출력
print(f"문서의 수: {len(data)}")
print("===" * 20)
print(f"[HUMAN]\n{question}\n")
print(f"[AI]\n{response}")

문서의 수: 705
[HUMAN]
기술스택은 python, sql, java, kubernates이고, 프로젝트는 학교웹사이트를 개발한 적이 있습니다. 데이터베이스 구축과 백엔드 개발을 담당했습니다. 추천 직업을 알려주세요.

[AI]
풀스택 개발자


In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/practice/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = loader = CSVLoader("../../data/rallit_text.csv", encoding='utf8')
docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a helpful assistant. 
            Answer the question based on the context below. You recommend a user's job.

            1. Determines the similarity of your resume to a saved resume file based on your resume. 
            2. It recommends 3 jobs based on the similarity. 
            3. It includes the percentage of similarity for each recommended job.
            4. The order of the job recommendations is provided in the order of the highest similarity. 

            \n\n
            {context}",
            """
        ),
        ("human", "{question}"),
    ]
)

chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)

result = chain.invoke("기술스택은 python, sql, java, kubernates이고, 프로젝트는 학교웹사이트를 개발한 적이 있습니다. 데이터베이스 구축과 백엔드 개발을 담당했습니다. 추천 직업을 알려주세요.")
print(result)

Created a chunk of size 688, which is longer than the specified 600
Created a chunk of size 635, which is longer than the specified 600
Created a chunk of size 689, which is longer than the specified 600
Created a chunk of size 792, which is longer than the specified 600
Created a chunk of size 652, which is longer than the specified 600
Created a chunk of size 636, which is longer than the specified 600
Created a chunk of size 674, which is longer than the specified 600
Created a chunk of size 726, which is longer than the specified 600
Created a chunk of size 646, which is longer than the specified 600
Created a chunk of size 920, which is longer than the specified 600
Created a chunk of size 652, which is longer than the specified 600
Created a chunk of size 4361, which is longer than the specified 600
Created a chunk of size 1000, which is longer than the specified 600
Created a chunk of size 645, which is longer than the specified 600
Created a chunk of size 845, which is longer t